### Import Package

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

### Bikin class, nentuin kolom dan baris & Batch Size

In [2]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32

img_rows,img_cols  : Ukuran array

Batch_size         : Semisal ada 1000 gambar dan batch_size=10, maka 1000 gambar/10 = 100 per epoch

In [3]:
train_data_dir = 'fer2013/train'
test_data = 'fer2013/test'

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

ImageDataGenerator : untuk membaca set data Anda dan melakukan augmentasi data

Augmentasi Data Gambar adalah teknik yang dapat digunakan untuk memperluas ukuran kumpulan data pelatihan secara artifisial dengan membuat versi gambar yang dimodifikasi dalam kumpulan data. 

Rescale adalah untuk mengubah skala nilai citra. Nilai 1/255 artinya setiap nilai akan dikali 1/255 sehingga nilainya akan berubah menjadi antara 0 dan 1

shear_range: Intensitas Geser (Sudut geser berlawanan arah jarum jam)

fill_mode: Ini digunakan untuk mengisi piksel setelah membuat perubahan orientasi gambar dengan metode yang digunakan

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_data,
    color_mode='grayscale',
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 24176 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.


shuffle: Untuk mengacak dataset untuk pelatihan yang lebih baik

In [6]:
model = Sequential()

Sequential         : Model Sequential adalah tumpukan linear lapisan, di mana kita dapat menggunakan berbagai macam lapisan yang tersedia di Keras. Lapisan yang paling umum adalah lapisan <b>Dense</b> yang merupakan lapisan neural network biasa yang terhubung dengan semua Weight dan bias yang sudah kita kenal.

https://keras.io/api/layers/#:~:text=Keras%20layers%20API,variables%20(the%20layer's%20weights).

Dropout: Dropout adalah teknik di mana neuron yang dipilih secara acak diabaikan selama pelatihan.

Flatten layer : Untuk meratakan output dari layer sebelumnya menjadi flat layer atau dengan kata lain dalam bentuk vektor

Dense  : di mana setiap neuron terhubung ke setiap neuron lainnya

In [7]:
# Blok 1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.2))

In [8]:
#Block 2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.2))

In [9]:
# Block 3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.2))

In [10]:
#Block 4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.2))

In [11]:
#Block 5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [12]:
# Block 6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [13]:
# Block 7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [15]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

monitor: Kuantitas untuk dipantau. Di sini saya memantau kehilangan validasi.

save_best_only=True, model terbaik terbaru menurut kuantitas yang dipantau tidak akan ditimpa.

erbose: int. 0: tetap, 1: perbarui pesan

min_delta: Perubahan minimum, jika perubahan mutlak kurang dari min_delta, akan dihitung sebagai tidak ada peningkatan.

patience : Jumlah epoch tanpa perbaikan setelah itu pelatihan akan dihentikan

restore_best_weights: Apakah akan mengembalikan bobot model dari epoch dengan nilai terbaik dari kuantitas yang dipantau. Jika Salah, bobot model yang diperoleh pada langkah terakhir pelatihan digunakan. 

faktor: di mana kecepatan belajar akan berkurang. new_lr = lr * faktor 

In [16]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)
earlyStop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                            factor=0.2,
                            patience=3,
                            verbose=1,
                            min_delta=0.0001)
callbacks = [earlyStop,checkpoint,reduce_lr]

In [17]:
model.compile(loss='categorical_crossentropy',
             optimizer= Adam(lr=0.001),
             metrics=['accuracy'])
nb_train_samples = 24176
nb_test_samples = 6043
epochs = 20
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_generator,
    validation_steps=nb_test_samples//batch_size)

C:\Users\LENOVO\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2968\893488696.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
755/755 [==============================] - ETA: 0s - loss: 1.8594 - accuracy: 0.2412
Epoch 1: val_loss improved from inf to 1.55457, saving model to Model.h5
755/755 [==============================] - 256s 337ms/step - loss: 1.8594 - accuracy: 0.2412 - val_loss: 1.5546 - val_accuracy: 0.2927 - lr: 0.0010
Epoch 2/20
755/755 [==============================] - ETA: 0s - loss: 1.5769 - accuracy: 0.2857
Epoch 2: val_loss improved from 1.55457 to 1.52888, saving model to Model.h5
755/755 [==============================] - 191s 253ms/step - loss: 1.5769 - accuracy: 0.2857 - val_loss: 1.5289 - val_accuracy: 0.3032 - lr: 0.0010
Epoch 3/20
755/755 [==============================] - ETA: 0s - loss: 1.5507 - accuracy: 0.3033
Epoch 3: val_loss improved from 1.52888 to 1.51049, saving model to Model.h5
755/755 [==============================] - 190s 252ms/step - loss: 1.5507 - accuracy: 0.3033 - val_loss: 1.5105 - val_accuracy: 0.3125 - lr: 0.0010
Epoch 4/20
755/755 [=====================

In [18]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

Haar Cascade adalah algoritma deteksi objek pembelajaran mesin yang digunakan untuk mengidentifikasi objek dalam gambar atau video 

In [19]:
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
classifier = load_model('EmotionDetectionModel.h5')

cv2.VideoCapture(0), 0 digunakan untuk menginstruksikan metode menggunakan webcam utama laptop.

In [20]:
class_labels=['Angry','Happy','Neutral','Sad','Surprise']
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    # membaca bingkai dari umpan kamera
    ret,frame=cap.read()
    labels=[]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # mendeteksi wajah dan mengembalikan daftar yang berisi koordinat
    faces=face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        # membuat persegi panjang menggunakan (x,y,w,h) dari satu wajah
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        # menyimpan gambar sebagai array
        roi_gray=gray[y:y+h,x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi=roi_gray.astype('float')/255.0
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)

            preds=classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position=(x,y)
            # menampilkan emosi yang terdeteksi 
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 20ms/step
